In [11]:
apikey = "Z4ehkkMbO1DlcukePiecqfxU55sRATGnDJHItl5B"

In [5]:
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

In [6]:
def solve_subsistence_problem(FoodNutrients,Prices,dietmin,dietmax,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    try: 
        p = Prices.apply(lambda x:x.magnitude)
    except AttributeError:  # Maybe not passing in prices with units?
        warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
        p = Prices

    p = p.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    Amin = Amin.reindex(dietmin.index,axis=0)
    idx = Amin.index.to_frame()
    idx['type'] = 'min'
    #Amin.index = pd.MultiIndex.from_frame(idx)
    #dietmin.index = Amin.index
    
    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index,axis=0)
    idx = Amax.index.to_frame()
    idx['type'] = 'max'
    #Amax.index = pd.MultiIndex.from_frame(idx)
    #dietmax.index = Amax.index

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([dietmin,
                   -dietmax]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='highs')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

In [7]:
SHEETs = [# Stigler's foods, modern prices
          ("https://docs.google.com/spreadsheet/ccc?key=1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A","Stigler Table B (2022 Prices)"),
         ]

In [8]:
import pandas as pd
%pip install eep153_tools
%pip install gnupg
from eep153_tools.sheets import read_sheets

df = read_sheets(SHEETs[0][0])[SHEETs[0][1]]
df

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Key available for students@eep153.iam.gserviceaccount.com.


,Food,Quantity,Units,Price,Date,Location,FDC
0,Wheat Flour,80.0,oz,3.79,[2022-2-28],Safeway,2432946
1,Wheat Cereal,16.4,oz,3.49,[2022-2-28],Safeway,2343964
2,Corn Meal,80.0,oz,4.49,[2022-2-28],Safeway,2079814
3,Rolled Oats,42.0,oz,3.99,[2022-2-28],Safeway,2478125
4,Evaporated Milk,12.0,oz,1.99,[2022-2-28],Safeway,1889123
5,Cabbage,1.0,lbs,1.29,[2022-2-28],Safeway,169975
6,Potatoes,1.0,lbs,1.49,[2022-2-28],Safeway,576920
7,Spinach,1.0,oz,0.25,[2022-2-28],Safeway,168462
8,Sweet Potatoes,1.0,lbs,1.99,[2022-2-28],Safeway,2427040
9,Navy Beans,1.0,lbs,3.49,[2022-2-28],Safeway,1859097


In [12]:
%pip install fooddatacentral
import fooddatacentral as fdc
import warnings

D = {}
count = 0
for food in  df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

FoodNutrients = pd.DataFrame(D,dtype=float)

Note: you may need to restart the kernel to use updated packages.


In [13]:
# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food',sort=False)['FDC Price'].min()

/srv/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


In [14]:
from eep153_tools.sheets import read_sheets

DRI_url = "https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/"

DRIs = read_sheets(DRI_url)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')

# Define *maximums*
diet_max = DRIs['diet_maximums'].set_index('Nutrition')

Key available for students@eep153.iam.gserviceaccount.com.


In [18]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for M 19-30 is $9.20 per day.


Diet (in 100s of grams or milliliters):
Rolled Oats      1.820657
Spinach          6.997015
Milk (Whole)    10.233686
Liver (Beef)     0.748359
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2407.785776          2400.0
Protein                           91.470404            56.0
Fiber, total dietary              33.600000            33.6
Folate, DFE                     1574.444808           400.0
Calcium, Ca                     2049.863704          1000.0
Carbohydrate, by difference      198.586576           130.0
Iron, Fe                          31.732150             8.0
Magnesium, Mg                    689.038827           400.0
Niacin                            16.000000            16.0
Phosphorus, P                   1666.070726           700.0
Potassium, K         

In [21]:
import pandas as pd

# Define the function to get DRIs based on the provided information
def get_dri(age, sex):
    """
    Returns a pandas.Series of Dietary Reference Intakes (DRIs) for the given age and sex.

    Parameters:
    age (int): The age of the person.
    sex (str): The sex of the person. Expected values: 'male' or 'female'.

    Returns:
    pandas.Series: A series of DRIs for various nutrients.
    """

    # DRI values based on the provided information
    dri_values = {
        'Protein_g': None,
        'Carbohydrate_g': None,
        'Carbohydrate_percent_kcal': None,
        'Dietary_Fiber_g': None,
        'Total_fat_percent_kcal': None,
        'Saturated_fat_percent_kcal': None
    }
    
    # Assigning values based on age and sex
    if age >= 1 and age <= 3:
        dri_values['Protein_g'] = 13 if sex == 'female' else 13
        dri_values['Dietary_Fiber_g'] = 14
    elif age >= 4 and age <= 8:
        dri_values['Protein_g'] = 19 if sex == 'female' else 19
        dri_values['Dietary_Fiber_g'] = 16.8 if sex == 'female' else 19.6
    elif age >= 9 and age <= 13:
        dri_values['Protein_g'] = 34 if sex == 'female' else 34
        dri_values['Dietary_Fiber_g'] = 22.4 if sex == 'female' else 25.2
    elif age >= 14 and age <= 18:
        dri_values['Protein_g'] = 46 if sex == 'female' else 52
        dri_values['Dietary_Fiber_g'] = 25.2 if sex == 'female' else 30.8
    elif age >= 19 and age <= 30:
        dri_values['Protein_g'] = 46 if sex == 'female' else 56
        dri_values['Dietary_Fiber_g'] = 28 if sex == 'female' else 33.6
    elif age >= 31 and age <= 50:
        dri_values['Protein_g'] = 46 if sex == 'female' else 56
        dri_values['Dietary_Fiber_g'] = 25.2 if sex == 'female' else 30.8
    elif age >= 51:
        dri_values['Protein_g'] = 46 if sex == 'female' else 56
        dri_values['Dietary_Fiber_g'] = 22.4 if sex == 'female' else 28
    
    # Carbohydrate and fat values are consistent across all age groups
    dri_values['Carbohydrate_g'] = 130  # Consistent for all age groups and sexes
    dri_values['Carbohydrate_percent_kcal'] = (45, 65)  # Consistent for all age groups and sexes
    dri_values['Total_fat_percent_kcal'] = (20, 35)  # Consistent for all age groups and sexes
    dri_values['Saturated_fat_percent_kcal'] = 10  # Consistent for all age groups and sexes

    # Convert the dictionary to a pandas Series
    dri_series = pd.Series(dri_values, name=f'DRIs for {age}-year-old {sex}')

    return dri_series

example_dri_series = get_dri(30, 'male')
example_dri_series

Protein_g                           56
Carbohydrate_g                     130
Carbohydrate_percent_kcal     (45, 65)
Dietary_Fiber_g                   33.6
Total_fat_percent_kcal        (20, 35)
Saturated_fat_percent_kcal          10
Name: DRIs for 30-year-old male, dtype: object